<a href="https://colab.research.google.com/github/ant358/tensorflow_projects/blob/main/NLP/TF_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:

url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

with open("./sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
# YOUR CODE HERE
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [ ]:

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size,
                        oov_token=oov_tok)

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,
                                maxlen=max_length,
                                padding=padding_type,
                                truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,
                                maxlen=max_length,
                                padding=padding_type,
                                truncating=trunc_type)

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

# Glove embedings
# glove.6B.100d.txt
# !gdown --id 1W5vZy2etitAblLdFn8_DxnsQKzfFJ98g

embeddings_index = {};
with open('./glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

# embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word);
#     if embedding_vector is not None:
#         embeddings_matrix[i] = embedding_vector;

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, 
                              embedding_dim, 
                              input_length=max_length,
                              # weights=[embeddings_matrix], 
                              # trainable=False
                              ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
num_epochs = 50

model.fit(training_padded,
            training_labels,
            epochs=num_epochs,
            validation_data=(testing_padded,
                            testing_labels),
            callbacks = [callback],
            verbose=2)
    

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1W5vZy2etitAblLdFn8_DxnsQKzfFJ98g
To: /content/glove.6B.100d.txt
100% 347M/347M [00:01<00:00, 257MB/s]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16016     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dense_

In [ ]:
sentences[:10]

In [ ]:
model.save("mymodel.h5")

In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    with open('sarcasm.json') as f:
        datas = json.load(f)

    sentences = []
    labels = []

    for data in datas:
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])
    
    # sentence, label -> train, validation split

    train_sentences = sentences[:training_size]
    train_labels = labels[:training_size]

    validation_sentences = sentences[training_size:]
    validation_labels = labels[training_size:]

    # sentence -> tokenizer
    tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(train_sentences)

    # tokenizer -> sequence
    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

    # sequence -> padded
    train_padded = pad_sequences(train_sequences, maxlen = max_length, truncating = trunc_type, padding = padding_type)
    validation_padded = pad_sequences(validation_sequences, maxlen = max_length, truncating = trunc_type, padding = padding_type)

    # label -> numpy array
    train_labels = np.array(train_labels)
    validation_labels = np.array(validation_labels)

    # Model
    model = Sequential([
                        Embedding(vocab_size, embedding_dim, input_length=max_length),
                        Bidirectional(LSTM(64, return_sequences = True)),
                        Bidirectional(LSTM(64, return_sequences = True)),
                        Bidirectional(LSTM(64)),
                        Dense(32, activation='relu'),
                        # Dense(32, activation='relu'),
                        Dense(16, activation='relu'),
                        # Dense(4, activation='relu'),
                        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    checkpoint_path = 'my_checkpoint_ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only = True,
                                 save_best_only = True,
                                 monitor = 'val_loss',
                                 verbose = 1)
    
    model.fit(train_padded, train_labels,
              validation_data = (validation_padded, validation_labels),
              callbacks = [checkpoint],
              epochs = 10)
    
    model.load_weights(checkpoint_path)

    return model
    

if __name__ == '__main__':
    model = solution_model()
    model.save("TF4-sarcasm.h5")
